In [157]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
import math

In [271]:
personal_data = pd.read_csv('./data/personal_train.csv')
personal_data = personal_data.drop(['Unnamed: 0'], axis=1)
other_data = pd.read_csv('./data/other_train.csv')
other_data = other_data.drop(['Unnamed: 0'], axis=1)

merged = pd.merge(personal_data,other_data,on=["name","address"])
duplicated = merged.duplicated(subset=["name","address"],keep=False)
duplicates = pd.DataFrame(columns=merged.columns)
indexes_to_drop = []
merged_without_duplicates = pd.DataFrame(columns=merged.columns)
for index, row in merged.iterrows():
    if(duplicated[index]):
        duplicates = duplicates.append(merged.iloc[index],ignore_index=True)
        indexes_to_drop.append(index)

duplicates.sort_values(["name","address"])



merged_without_duplicates = merged

for index in indexes_to_drop:
    merged_without_duplicates = merged_without_duplicates.drop(merged.index[index])

merged = merged_without_duplicates

def checkNaN(arr):
    for item in arr:
        if(type(item) is not float or ~np.isnan(item)):
            return False
    return True

def checkQuestionMark(arr):
    for item in arr:
        if(item is not '?'):
            return False
    return True

def nanOrQuestionMark(arr):
    for item in arr:
        if(item is not '?' and ((type(item) is not np.float64 and type(item) is not float) or not np.isnan(item))):
            return False
    return True

def filter_nan_question_mark(arr):
    result = []
    for item in arr:
        if(item is not '?' and ((type(item) is not np.float64 and type(item) is not float) or not np.isnan(item))):
            result.append(item)
    return result        

def getUniqueRow(startIndex,endIndex):
    row_dict = {}

    for column in duplicates.columns: 
        temp = []
        for index in range(startIndex,endIndex):
            temp.append(duplicates.iloc[index][column])
         
        if(checkNaN(temp)):
            row_dict[column] = np.nan
        elif(checkQuestionMark(temp)):
            row_dict[column] = '?'
        elif(nanOrQuestionMark(temp)):
            row_dict[column] = '?'
        else:
            temp = filter_nan_question_mark(temp)
            value = randint(0,len(temp)-1)
            row_dict[column] = temp[value]
    return row_dict
    
count = 0
lastDuplicate = duplicates.iloc[0]["name"]
columns = duplicates.columns.values
for index,row in duplicates.iterrows():
    entry = duplicates.iloc[index]
    name = entry["name"]
    if(name == lastDuplicate):
        count += 1
    else:
        merged = merged.append(getUniqueRow(index-count,index),ignore_index=True)
        lastDuplicate = name
        count = 1
        
    if(len(duplicates) - 1 == index):
        merged = merged.append(getUniqueRow(index-count+1,index + 1),ignore_index=True)
        
        
    